<a href="https://colab.research.google.com/github/gkdivya/EVA/blob/main/1_NeuralArchitecture/MNIST_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Understanding Receptive field and CNN parameters using MNIST

In this notebook, lets try understanding more about receptive field and CNN params

## Loading PyTorch libraries

PyTorch is an open source machine learning library based on the Torch library, used for applications such as computer vision and natural language processing, primarily developed by Facebook's AI Research lab.

Below packages are mainly imported while developing models in PyTorch

Package  | Description
--- | ---
torch	 | The top-level PyTorch package and tensor library.
torch.nn  |	A subpackage that contains modules and extensible classes for building neural networks.
torch.autograd  |	A subpackage that supports all the differentiable Tensor operations in PyTorch.
torch.nn.functional  |	A functional interface that contains typical operations used for building neural networks like loss functions, activation functions, and convolution operations.
torch.optim |	A subpackage that contains standard optimization operations like SGD and Adam.
torch.utils |	A subpackage that contains utility classes like data sets and data loaders that make data preprocessing easier.
torchvision |	A package that provides access to popular datasets, model architectures, and image transformations for computer vision.



In [ ]:
from __future__ import print_function #will allow to use print as a function, they must be at the top of the file
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

## Defining model class 'Net'

We are defining class 'Net', creating instances of convolution, pooling in constructor.

**Convolution :** 
Its a mathematical operation where a kernel mostly 3*3 matrix slides over a 2D image data performing a multiplication and addition of the sliding window results.

![Convolution](https://miro.medium.com/max/535/1*Zx-ZMLKab7VOCQTxdZ1OAw.gif)

In PyTorch, we use nn.conv2d to perform the operation.


**Pooling :**
The maximum value in 2*2 sliding window is picked up.

![Max Pooling](https://nico-curti.github.io/NumPyNet/NumPyNet/images/maxpool.gif)


**Activation Function :**
Relu activation function is used where only the positive values are considered, negative values will be taken as zero



In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)       #Input: 28*28*1    Output:28 * 28 * 32    GRF:3 * 3  (GRF - Global Receptive Field)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)      #Input: 28*28*32   Output:28 * 28 * 64    GRF:5 * 5
        self.pool1 = nn.MaxPool2d(2, 2)                   #Input: 28*28*64   Output:14 * 14 * 64    GRF:10*10 (for now we are considering that with max pooling, receptive field doubles - but this is not entirely correct)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)     #Input: 14*14*64   Output:14 * 14 * 128   GRF:12*12
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)    #Input: 14*14*128  Output:14 * 14 * 256   GRF:14*14
        self.pool2 = nn.MaxPool2d(2, 2)                   #Input: 14*14*256  Output: 7 * 7 * 256    GRF:28*28
        self.conv5 = nn.Conv2d(256, 512, 3)               #Input: 7*7*256    Output: 5 * 5 * 12     GRF:30*30
        self.conv6 = nn.Conv2d(512, 1024, 3)              #Input: 5*5*12     Output: 3 * 3 * 1024   GRF:32*32
        self.conv7 = nn.Conv2d(1024, 10, 3)               #Input: 3*3*1024   Output: 1 * 1 * 10     GRF:34*34

    def forward(self, x):
        #The first convolutional layer self.conv1 has a convolutional operation on input tensor x, followed by a relu activation operation 
        #whcih is then passed to second convolution operation self.conv2 followed by a relu whose output is then passed to a max pooling 
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))

        #The output of the max pool operation is passed to another two convolution and relu activation operation followed by max pooling. 
        #The relu() and the max_pool2d() calls are just pure operations. Neither of these have weights  
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))

        #The output of the max pool operation from 4th convolution is passed to another two sets of convolution and relu activation operation
        x = F.relu(self.conv6(F.relu(self.conv5(x))))

        # The output from 6th convolution is then passed to a final convolution followed by a relu operation
        x = F.relu(self.conv7(x))

        #The flatten's all of the tensor's elements into a single dimension.
        x = x.view(-1, 10)

        # Inside the network we usually use relu() as our non-linear activation function, but for the output layer, whenever we have a single category that we are trying to predict, we use softmax(). 
        #The softmax function returns a positive probability for each of the prediction classes, and the values sum to 1.
        return F.log_softmax(x)

## Model Summary

torchsummary - Package used to display model's summary

CUDA® is a parallel computing platform and programming model developed by NVIDIA for general computing on graphical processing units (GPUs). With CUDA, developers are able to dramatically speed up computing applications by harnessing the power of GPUs.

In [ ]:
!pip install torchsummary
from torchsummary import summary

#This command will return a boolean (True/False) based on the GPU availability.
use_cuda = torch.cuda.is_available()                   #Checking if GPU is available

#use the use_cuda flag to specify which device you want to use
# This will set the device to the GPU if one is available and to the CPU if there isn’t a GPU available. This means that you don’t need to hard code changes into your code to use one or the other.
device = torch.device("cuda" if use_cuda else "cpu") 

#move the model to the choosen device
model = Net().to(device)      

# prints the model summary
summary(model, input_size=(1, 28, 28))                #Gray scale images hence its only 1 channel

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
            Conv2d-2           [-1, 64, 28, 28]          18,496
         MaxPool2d-3           [-1, 64, 14, 14]               0
            Conv2d-4          [-1, 128, 14, 14]          73,856
            Conv2d-5          [-1, 256, 14, 14]         295,168
         MaxPool2d-6            [-1, 256, 7, 7]               0
            Conv2d-7            [-1, 512, 5, 5]       1,180,160
            Conv2d-8           [-1, 1024, 3, 3]       4,719,616
            Conv2d-9             [-1, 10, 1, 1]          92,170
Total params: 6,379,786
Trainable params: 6,379,786
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 1.51
Params size (MB): 24.34
Estimated Total Size (MB): 25.85
-------------------------------------

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


## Model Training


*   **torch.manual_seed** used to set seed to generate random numbers. This is to ensure same random set of images will be chosen for batch based on seed to observe similar accuracy every time [PyTorch Documentation](https://pytorch.org/docs/stable/generated/torch.manual_seed.html)

*   **Batch size** is set as 128

*   **num_workers** Having more workers will increase the memory usage and that’s the most serious overhead. I’d just experiment and launch approximately as many as are needed to saturate the training.

  num_workers equal 0 means that it’s the main process that will do the data loading when needed, num_workers equal 1 is the same as any n, but you’ll only have a single worker, so it might be slow

*  **pin_memory:** If you load your samples in the Dataset on CPU and would like to push it during training to the GPU, you can speed up the host to device transfer by enabling pin_memory.
This lets your DataLoader allocate the samples in page-locked memory, which speeds-up the transfer.

* **Transforms:** Transforms are common image transformations. They can be chained together using Compose. Additionally, there is the torchvision.transforms.functional module. Functional transforms give fine-grained control over the transformations. [Transforms](https://pytorch.org/vision/stable/transforms.html)

*   Images are normalized using mean and std values of MNIST dataset: **0.1307, 0.3081**
[MNIST Normalization Reference](https://discuss.pytorch.org/t/normalization-in-the-mnist-example/457)



In [ ]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

#load train data and normalize the pixel values with mean and std computed on the MNIST training set
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)

#load test data and normalize the pixel values with mean and std computed on the MNIST training set
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)



Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw




Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw




Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw

Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


## Model Training
Steps for training is defined below:

- forward pass : compute predicted outputs by passing inputs to the model
- calculate the loss
- Backward pass
- perform single optimization step (parameter update)
- Clear the gradients for all optimized variables
- update average training loss

**Test the trained Network**

Finally, we test our best model on previously unseen test data and evaluate its performance. Testing on unseen data is good way to check our model perforamance to see if our model generalizes well. It may be useful to be granular in this analysis and take a look at how this model performs on each class as well as looking at its overall loss and accuracy

In [ ]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 2):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

loss=1.8367058038711548 batch_id=468: 100%|██████████| 469/469 [00:17<00:00, 26.29it/s]



Test set: Average loss: 1.6904, Accuracy: 3857/10000 (39%)



## Analysis:

We have observed 39% accuracy in above model. Problem in above model is because of using activation function relu just before prediction. Negative weights also play an important role with the prediction

## Second Iteration - Without relu before last convolution

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)       #Input: 28*28*1  Output:26 * 26 * 32 GRF:3 * 3 
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)      #Input: 26*26*32 Output:24 * 24 * 64 GRF:5 * 5
        self.pool1 = nn.MaxPool2d(2, 2)                   #Input: 24*24*64 Output:12 * 12 * 64 GRF:10*10
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)     #Input: 28*28*1  Output:26 * 26 * 32 GRF:12*12
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)    #Input: 28*28*1  Output:26 * 26 * 32 GRF:14*14
        self.pool2 = nn.MaxPool2d(2, 2)                   #Input: 28*28*1  Output:26 * 26 * 32 GRF:28*28
        self.conv5 = nn.Conv2d(256, 512, 3)               #Input: 28*28*1  Output:26 * 26 * 32 GRF:30*30
        self.conv6 = nn.Conv2d(512, 1024, 3)              #Input: 28*28*1  Output:26 * 26 * 32 GRF:32*32
        self.conv7 = nn.Conv2d(1024, 10, 3)               #Input: 28*28*1  Output:26 * 26 * 32 GRF:34*34

    def forward(self, x):
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        x = F.relu(self.conv6(F.relu(self.conv5(x))))
        x = self.conv7(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [ ]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
            Conv2d-2           [-1, 64, 28, 28]          18,496
         MaxPool2d-3           [-1, 64, 14, 14]               0
            Conv2d-4          [-1, 128, 14, 14]          73,856
            Conv2d-5          [-1, 256, 14, 14]         295,168
         MaxPool2d-6            [-1, 256, 7, 7]               0
            Conv2d-7            [-1, 512, 5, 5]       1,180,160
            Conv2d-8           [-1, 1024, 3, 3]       4,719,616
            Conv2d-9             [-1, 10, 1, 1]          92,170
Total params: 6,379,786
Trainable params: 6,379,786
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 1.51
Params size (MB): 24.34
Estimated Total Size (MB): 25.85
-------------------------------------

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [ ]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [ ]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 2):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

loss=0.03109382838010788 batch_id=468: 100%|██████████| 469/469 [00:18<00:00, 24.82it/s]



Test set: Average loss: 0.0678, Accuracy: 9774/10000 (98%)



**Conclusion:**

Relu should not be used before last layer. Negative weights will help with the prediction. Just by removing relu model accuracy increased to 98%